In [56]:
import pickle
import pandas as pd

In [57]:
te= pd.read_csv('/opt/ml/input/data/new_test_data.csv')
tr= pd.read_csv('/opt/ml/input/data/new_train_data.csv')

In [33]:
with open('/opt/ml/input/code/dkt/dkt/lightgcn_for_tag/tag2index.pickle', 'rb') as fr:
    id2idx = pickle.load(fr)

In [34]:
len(id2idx)

8354

In [37]:
tt.userID.nunique() + 912

8354

In [6]:
tr=pd.read_csv('/opt/ml/input/data/train_data.csv')
te= pd.read_csv('/opt/ml/input/data/test_data.csv')

In [8]:
tt= pd.concat([tr,te], axis=0)

In [9]:
tt.KnowledgeTag.nunique()

912

In [21]:
tr.KnowledgeTag.nunique()

912

In [22]:
tt.userID.nunique()

7442

In [23]:
tr.userID.nunique()

6698

In [15]:
tt= tt.astype({'KnowledgeTag':'str'})

In [16]:
tt.KnowledgeTag

0         7224
1         7225
2         7225
3         7225
4         7225
          ... 
260109    8832
260110    8832
260111    8244
260112    8244
260113    8832
Name: KnowledgeTag, Length: 2526700, dtype: object

In [17]:
sum(tr.answerCode==-1)

0

In [18]:
sum(te.answerCode==-1)

744

In [31]:
tt.drop_duplicates(
        subset=["userID", "KnowledgeTag"], keep="last")

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224
5,0,A060001007,A060000001,1,2020-03-24 00:17:47,7225
12,0,A060003007,A060000003,1,2020-03-26 05:54:11,7226
18,0,A060005006,A060000005,0,2020-03-31 05:05:14,7227
19,0,A060005007,A060000005,1,2020-03-31 05:05:48,7228
...,...,...,...,...,...,...
260097,7417,A050193004,A050000193,-1,2020-09-06 13:09:15,10402
260102,7439,A040003005,A040000003,0,2020-01-14 08:39:48,7268
260108,7439,A040197006,A040000197,1,2020-08-21 07:39:45,2132
260112,7439,A040130004,A040000130,1,2020-10-14 23:09:31,8244


In [48]:
import time
from datetime import datetime

In [49]:
def convert_time(s):
    timestamp = time.mktime(
        datetime.strptime(s, "%Y-%m-%d %H:%M:%S").timetuple()
    )
    return int(timestamp)

tr["Timestamp"] = tr["Timestamp"].apply(convert_time)

In [50]:
tr

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,1585009031,7224
1,0,A060001002,A060000001,1,1585009034,7225
2,0,A060001003,A060000001,1,1585009042,7225
3,0,A060001004,A060000001,1,1585009049,7225
4,0,A060001005,A060000001,1,1585009056,7225
...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,1591339821,438
2266582,7441,A040165001,A040000165,1,1597971999,8836
2266583,7441,A040165002,A040000165,1,1597972010,8836
2266584,7441,A040165003,A040000165,1,1597972056,8836


In [51]:
tr['diff'] =tr.sort_values(['userID','Timestamp']).groupby('userID')['Timestamp'].diff(periods=-1).apply(abs)

In [55]:
tr['diff'] > 0

0           True
1           True
2           True
3           True
4           True
           ...  
2266581     True
2266582     True
2266583     True
2266584     True
2266585    False
Name: diff, Length: 2266586, dtype: bool

In [53]:
user_start_pos = tr['diff'] > 0
train_df_temp['previous_answer_count'] = train_df_temp.groupby('userID')['answerCode'].cumsum().shift(fill_value=0)
train_df_temp['shift_previous_answer_count'] = train_df_temp.groupby('userID')['answerCode'].cumsum().shift(fill_value=0)
train_df_temp.loc[user_start_pos, ['previous_answer_count', 'shift_previous_answer_count']] = 0
train_df_temp['shift_previous_answer_count'] = train_df_temp['shift_previous_answer_count'].shift(shift_size)
train_df_temp['temp'] = len(train_df_temp) * [1]
train_df_temp['previous_problem_count'] = train_df_temp.groupby('userID')['temp'].cumsum().shift(fill_value=0)
train_df_temp['previous_problem_count'] = train_df_temp['previous_problem_count'].apply(lambda x: shift_size if x > shift_size else x)
train_df_temp.loc[user_start_pos, ['previous_problem_count']] = 0

train_df_temp['shift_previous_answer_count'] = train_df_temp.apply(lambda x: 0 if x['previous_problem_count'] < shift_size else x['shift_previous_answer_count'], axis=1)
train_df_temp['count'] = train_df_temp['previous_answer_count'] - train_df_temp['shift_previous_answer_count']
train_df['user_recent_acc'] = (train_df_temp['count'] / train_df_temp['previous_problem_count']).fillna(0)

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,diff
0,0,A060001001,A060000001,1,1585009031,7224,3.0
1,0,A060001002,A060000001,1,1585009034,7225,8.0
2,0,A060001003,A060000001,1,1585009042,7225,7.0
3,0,A060001004,A060000001,1,1585009049,7225,7.0
4,0,A060001005,A060000001,1,1585009056,7225,11.0
...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,1591339821,438,6632178.0
2266582,7441,A040165001,A040000165,1,1597971999,8836,11.0
2266583,7441,A040165002,A040000165,1,1597972010,8836,46.0
2266584,7441,A040165003,A040000165,1,1597972056,8836,73.0
